In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import json
import seaborn as sns
import re
from datetime import timedelta

%matplotlib inline

In [2]:
#importing all tables from data folder
members = pd.read_csv('NSS_GreatCare_Members.csv')
admissions = pd.read_csv('NSS_Hospital_Inpatient_Admissions.csv')
pcp = pd.read_csv('NSS_PrimaryCare_Office_Visits.csv', low_memory = False)

2. What percent of patients have a PCP visit within a week of discharge?

In [3]:
#converting admissions dates to datetime for calculations
admissions[['Encounter Start Date','Encounter End Date']] = admissions[['Encounter Start Date','Encounter End Date']].apply(pd.to_datetime)

#converting pcp dates to datetime for calculations
pcp['Encounter Date'] = pd.to_datetime(pcp['Encounter Date'])

#selecting relevant columns from each data frame, then joining hospital admissions to pcp care
a_df = admissions[['Encounter ID', "Patient ID", 'Encounter Start Date', "Encounter End Date"]]
pcp_df = pcp[['Encounter ID', 'Patient ID', 'Encounter Date']]

#merging relevant admissions info to relevant pcp visits
adpcp_df = a_df.merge(pcp_df, how='left', on='Patient ID')

#creating new column for 7 days, to find out which patients had a PCP visit after a week from discharge
adpcp_df['7_days'] = adpcp_df['Encounter Date'] - adpcp_df['Encounter End Date']
adpcp_df

,Encounter ID_x,Patient ID,Encounter Start Date,Encounter End Date,Encounter ID_y,Encounter Date,7_days
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1228384|1043427248|1|SHIFTED,2015-12-08,1058 days
1,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1326684|1043427248|1|SHIFTED,2012-06-18,-210 days
2,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|155317|1043427248|1|SHIFTED,2014-06-10,512 days
3,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|439093|1043427248|1|SHIFTED,2014-09-28,622 days
4,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|711310|1043427248|1|SHIFTED,2014-03-16,426 days
...,...,...,...,...,...,...,...
44093,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|1238517|1407058787|2,2015-12-05,8 days
44094,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|302749|1407058787|2,2015-12-13,16 days
44095,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|781849|1205884392|1,2014-05-18,-558 days
44096,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|93182|1043427248|1,2013-07-20,-860 days


In [4]:
patients7 = []

for i in adpcp_df['7_days']:
    if (i <= dt.timedelta(days=7)) and (i > dt.timedelta(days=0)) :
        patients7.append('yes')
    else:
        patients7.append('no')

In [5]:
adpcp_df['7_days'] = patients7

In [6]:
adpcp_df['7_days'].value_counts()

no     41736
yes     2362
Name: 7_days, dtype: int64

In [7]:
# To finish Q2, I will filter out all of the 'no' and be left with only 'yes'. I can merge that back to the original 'Inpatient' DF to find a %

y = adpcp_df.loc[adpcp_df['7_days'] == 'yes']

# Using this filtered df, I will remove duplicates mentioned in my Markdown above. This way, each hospital encounter will only count 1 time if they visit a doc after.
y = y.drop_duplicates(subset='Encounter ID_x', keep='first')
y

,Encounter ID_x,Patient ID,Encounter Start Date,Encounter End Date,Encounter ID_y,Encounter Date,7_days
28,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,2012-06-13,2012-06-21,COV|PBSL|DS1|10073|P|301709|1043427248|1|SHIFTED,2012-06-27,yes
50,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,2012-12-06,2012-12-09,COV|PCSL|DS1|10101|P|1189971|1043427248|1,2012-12-12,yes
231,HIA|DS1|10303|DS1|NPI|1710918545|1,DS1|10303,2012-06-21,2012-06-24,COV|PBSL|DS1|10303|P|10896|1912987553|1,2012-06-30,yes
247,HIA|DS1|10305|DS1|NPI|1184722779|1,DS1|10305,2014-05-10,2014-05-12,COV|PBSL|DS1|10305|P|823298|1184722779|1,2014-05-13,yes
250,HIA|DS1|10325|DS1|NPI|1184722779|1,DS1|10325,2012-07-06,2012-07-07,COV|PBSL|DS1|10325|P|490171|1184722779|1,2012-07-08,yes
...,...,...,...,...,...,...,...
44039,HIA|DS1|9943|DS1|NPI|1184722779|3,DS1|9943,2014-10-28,2014-10-30,COV|PCSL|DS1|9943|P|1315984|1184722779|1,2014-11-03,yes
44051,HIA|DS1|9945|DS1|NPI|1902803315|1,DS1|9945,2013-02-09,2013-02-11,COV|PCSL|DS1|9945|P|840374|1043427248|1,2013-02-14,yes
44067,HIA|DS1|9978|DS1|NPI|1427055839|1,DS1|9978,2015-09-01,2015-09-05,COV|PCSL|DS1|9978|P|696226|1043427248|1,2015-09-11,yes
44071,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,2015-02-06,2015-02-11,COV|PCSL|DS1|9979|P|281365|1477666352|1,2015-02-15,yes


In [8]:
# Need to clean up my columns to get ready to merge

w = y[['Encounter ID_x','Encounter Date','7_days']]
w

,Encounter ID_x,Encounter Date,7_days
28,HIA|DS1|10073|DS1|NPI|1902803315|1,2012-06-27,yes
50,HIA|DS1|10101|DS1|NPI|1427055839|1,2012-12-12,yes
231,HIA|DS1|10303|DS1|NPI|1710918545|1,2012-06-30,yes
247,HIA|DS1|10305|DS1|NPI|1184722779|1,2014-05-13,yes
250,HIA|DS1|10325|DS1|NPI|1184722779|1,2012-07-08,yes
...,...,...,...
44039,HIA|DS1|9943|DS1|NPI|1184722779|3,2014-11-03,yes
44051,HIA|DS1|9945|DS1|NPI|1902803315|1,2013-02-14,yes
44067,HIA|DS1|9978|DS1|NPI|1427055839|1,2015-09-11,yes
44071,HIA|DS1|9979|DS1|NPI|1689772592|1,2015-02-15,yes


In [9]:
# Merge admissions to my new clean set of patients who were admitted after 7 days.

doc_7_day = admissions.merge(w, how = 'left', left_on = 'Encounter ID', right_on = 'Encounter ID_x')
doc_7_day

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,Encounter ID_x,Encounter Date,7_days
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,NaN,NaT,NaN
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,NaN,NaT,NaN
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,HIA|DS1|10073|DS1|NPI|1902803315|1,2012-06-27,yes
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,HIA|DS1|10101|DS1|NPI|1427055839|1,2012-12-12,yes
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,...,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,HIA|DS1|9979|DS1|NPI|1689772592|1,2015-02-15,yes
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,...,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,HIA|DS1|9981|DS1|NPI|1427055839|1,2014-08-11,yes
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,...,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,NaN,NaT,NaN
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,...,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,NaN,NaT,NaN


In [10]:

# 1864(doc visits) out of the total 6213(hospital visits). 30.00% went to their PCP within 7 days after the hospital discharge.

print(doc_7_day['7_days'].value_counts())
print(1864/6213*100)

yes    1857
Name: 7_days, dtype: int64
30.001609528408174


BONUS 5
Which individual or PCP groups should we make an extra effort with to build a relationship? Which have the highest volume?

In [11]:
#finding Provier Group NPI's and counting should provide which facility saw the most patients?? 
#rationale here is that Encounter ID's are unique, so each duplicated NPI should be 1 visit?? 
b1 = pcp[['Encounter ID','Patient ID', 'Provider Group NPI', 'Encounter Date']]
b1['Provider Group NPI'] = b1['Provider Group NPI'].astype('object')
b1['Provider Group NPI'].value_counts()
#make this cleaner somehow to see full PGNPI number

<ipython-input-11-cc73b1f829e2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b1['Provider Group NPI'] = b1['Provider Group NPI'].astype('object')


1.043427e+09    114849
1.184723e+09     44030
1.164513e+09     18096
1.407059e+09     12041
1.316025e+09      8686
                 ...  
1.962566e+09         1
1.083750e+09         1
1.639473e+09         1
1.528052e+09         1
1.699021e+09         1
Name: Provider Group NPI, Length: 380, dtype: int64

BONUS 5. 
What percent of our patients do not have a PCP assigned or haven't seen their PCP recently?

In [12]:
#adding searchable text 'no pcp' to replace blank values in the Provider Group NPI column
b1['Provider Group NPI'] = b1['Provider Group NPI'].fillna('no pcp')

#trying to find how many patients do not have a PCP .. 
no_pcp = []
for index, row in b1.iterrows():
    if row['Provider Group NPI'] == 'no pcp':
        no_pcp.append('No PCP')
    else:
        no_pcp.append('Has PCP')

<ipython-input-12-20e96d26865f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b1['Provider Group NPI'] = b1['Provider Group NPI'].fillna('no pcp')


In [21]:
#3,729 patients do not have a PCP.. drop duplicates because each unique patient id was counted for every itterow calculation?
# seems really low if dupes are dropped.. .
yn = pd.DataFrame(no_pcp)
yn = yn.rename(columns={yn.columns[0]: 'PCP Y/N'})
yn.value_counts()
#yn.drop_duplicates()
#yn

PCP Y/N
Has PCP    227644
No PCP       3729
dtype: int64

In [23]:
print(557/6213*100)
print('dropping dupes')
print(3729/6213*100)
print('not dropping dupes')

8.965073233542572
dropping dupes
60.01931434089812
not dropping dupes


BONUS 6.
Are there certain patients we should focus on (e.g. frequent fliers)?

In [15]:
#displaying all rows, columns, etc.. full list .. seems this code has to be placed before the syntax you're looking for.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

a = admissions[['Patient ID', 'Attending Provider NPI']]
p = pcp[['Patient ID', 'Provider Group NPI']]
ap = pd.merge(a,p, how='left', on='Patient ID')
ap.columns=['Patient ID', 'Hospital Visits', 'PCP Visits']
ap.value_counts('Patient ID') #counting patient ID patient 3812, 57685, and 53241 have the most occurances.

Patient ID
DS1|3812     544
DS1|57685    312
DS1|53241    247
DS1|24041    220
DS1|45849    220
DS1|60619    216
DS1|33966    200
DS1|37477    198
DS1|47885    192
DS1|10734    185
DS1|10451    168
DS1|60502    165
DS1|11089    162
DS1|6506     140
DS1|22395    140
DS1|18787    130
DS1|14435    125
DS1|12651    119
DS1|16151    115
DS1|34414    108
DS1|33880    105
DS1|52751    105
DS1|5859     105
DS1|61015    104
DS1|55342     95
DS1|4652      90
DS1|32814     90
DS1|11476     90
DS1|39480     90
DS1|21571     88
DS1|23411     88
DS1|25562     87
DS1|17710     85
DS1|11581     85
DS1|21902     84
DS1|31599     84
DS1|23276     84
DS1|13892     81
DS1|22806     80
DS1|3663      80
DS1|50174     80
DS1|35155     80
DS1|55122     78
DS1|11785     77
DS1|40749     76
DS1|30988     75
DS1|26751     75
DS1|21011     72
DS1|54665     72
DS1|39734     72
DS1|5182      72
DS1|51692     72
DS1|6158      70
DS1|14209     70
DS1|4639      69
DS1|15935     68
DS1|47312     68
DS1|43857     68
DS1

In [16]:
ap.groupby('Patient ID')['Hospital Visits'].value_counts()

Patient ID  Hospital Visits
DS1|10      1.962461e+09       10
DS1|10033   1.417171e+09       12
DS1|10043   1.669599e+09       12
DS1|10073   1.407881e+09       16
DS1|10101   1.407009e+09       16
DS1|10118   1.518046e+09        6
DS1|10126   1.346557e+09       27
DS1|10130   1.316989e+09        6
DS1|10132   1.417995e+09        1
DS1|10135   1.396818e+09        5
DS1|10168   1.447495e+09        4
DS1|10169   1.528163e+09       36
DS1|10191   1.366605e+09        2
DS1|10193   1.427376e+09       18
DS1|10228   1.619074e+09       18
DS1|10239   1.972527e+09       23
DS1|10244   1.366847e+09        6
DS1|10253   1.346566e+09        2
            1.750346e+09        2
            1.932127e+09        2
DS1|10291   1.598994e+09        7
            1.720241e+09        7
DS1|10296   1.023149e+09        1
DS1|10303   1.508842e+09       14
DS1|10305   1.730127e+09        5
DS1|10325   1.053334e+09        4
DS1|10343   1.922324e+09        7
DS1|10349   1.700829e+09        4
DS1|10375   1.982865

In [17]:
ap.groupby('Patient ID')['PCP Visits'].value_counts()

Patient ID  PCP Visits  
DS1|10      1.144213e+09      5
            1.083695e+09      2
            1.316025e+09      2
            1.639169e+09      1
DS1|10033   1.043427e+09     12
DS1|10043   1.043427e+09     12
DS1|10073   1.043427e+09     16
DS1|10101   1.043427e+09     16
DS1|10118   1.043427e+09      6
DS1|10126   1.043427e+09     27
DS1|10130   1.043427e+09      6
DS1|10135   1.184723e+09      3
            1.043427e+09      2
DS1|10168   1.184723e+09      4
DS1|10169   1.639169e+09     28
            1.144213e+09      4
            1.316025e+09      2
            1.518947e+09      1
            1.962483e+09      1
DS1|10191   1.043427e+09      2
DS1|10193   1.043427e+09     18
DS1|10228   1.043427e+09     18
DS1|10239   1.184723e+09     17
            1.043427e+09      6
DS1|10244   1.184723e+09      3
            1.407059e+09      3
DS1|10253   1.164513e+09      6
DS1|10291   1.043427e+09     12
            1.184723e+09      2
DS1|10296   1.043427e+09      1
DS1|10303   1.9